# Projeto 4 - Pegando todos os setores da bolsa no site da B3

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
import zipfile
import os

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'

In [ ]:
driver.get(url)

In [ ]:
local_botao_expandir = '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[1]/div/div/a/i'

In [ ]:
botao_expandir = driver.find_element('xpath', local_botao_expandir)

### O site da B3 é só uma casca da antiga BVMF. 

Nesses casos, precisamos trocar pro iframe que importa. 

In [ ]:
WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "bvmf_iframe")))

In [ ]:
botao_expandir = driver.find_element('xpath', local_botao_expandir)

In [ ]:
#botao_expandir.click()
driver.execute_script("arguments[0].click();", botao_expandir)

In [ ]:
planilha = driver.find_element('xpath', '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[2]/div/app-companies-home-filter-classification/form/div[2]/div[3]/div[2]/p/a')

driver.execute_script("arguments[0].click();", planilha)

#time.sleep(5) #tem que esperar baixar a planilha 

In [ ]:
driver.quit()

### Agora vamos extrair o ZIP

In [ ]:
caminho_downloads = r'C:\Users\lsiqu\Downloads'

In [ ]:
arquivo_zip = zipfile.ZipFile(caminho_downloads + r"\ClassifSetorial.zip")

for planilha in arquivo_zip.namelist():

    setores = pd.read_excel(arquivo_zip.open(planilha), skiprows=6)

In [ ]:
arquivo_zip.close()

In [ ]:
os.remove(caminho_downloads + r"\ClassifSetorial.zip")

In [ ]:
setores

In [ ]:
setores['SUBSETOR'] = setores['SUBSETOR'].ffill()
setores = setores.dropna(subset = ['SEGMENTO'])
setores = setores[['SUBSETOR', 'SEGMENTO', 'LISTAGEM']]

setores

In [ ]:
setores = setores.dropna()
setores.columns = ['SETOR', 'NOME', 'TICKER']
setores

In [ ]:
setores.to_csv("setores.csv", index = False)